In [1]:
# reload modules
%load_ext autoreload
%autoreload 2

In [3]:
import os

# os.chdir("") # set the root directory of the project
import numpy as np
import pandas as pd
from sra.dataloader import (
    get_covs_runs,
    get_run_paths,
    get_trial_paths,
    get_covs_trial,
)
from sra.cfggen import parse_ll, parse_dotfiles, gen_cfg_inter
from sra.mapper import get_bbs_fuzz, get_mapping, fuzzdat_to_obs
from sra.estimator import Graph
from sra.estimator_chunk import structure_estimation


In [4]:
def get_statistics(num_arr: np.ndarray):
    mean_val = np.mean(num_arr)
    std_val = np.std(num_arr)
    min_val = np.min(num_arr)
    max_val = np.max(num_arr)
    quantiles = pd.DataFrame(num_arr).quantile(
        [0.01] + [1 / 20 * i for i in range(1, 20)] + [0.99]
    )
    print(f"mean: {mean_val}, std: {std_val}, min: {min_val}, max: {max_val}")
    print(f"quantiles:\n{quantiles}")


In [5]:
import threading

from functools import wraps
import errno
import os
import signal


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


@timeout(5)
def print_result(observable_nodes, graph, target_node):
    print(
        f"{target_node}: {structure_estimation(np.zeros((1, len(observable_nodes))),graph,target_node,observable_nodes,2)}"
    )

# Schedule2

In [6]:
source_dir = "fuzz-data/ft_data/source_code/schedule2"
source_name = "schedule2_fuzz"

dotfiles = [
    fname
    for fname in os.listdir(source_dir)
    if fname.startswith(".") and fname.endswith(".dot")
]
functions = sorted([fname.split(".")[1] for fname in dotfiles])

bbs_fuzz_path = "fuzz-data/schedule2_exp_1/schedule2_01/schedule2_aflpp_run_01/ft_schedule2.json"
bbs_fuzz = get_bbs_fuzz(bbs_fuzz_path)

debug_info_dict, blocks_dict = parse_ll(
    "fuzz-data/ft_data/source_code/schedule2/schedule2_fuzz.ll"
)
cfgs_intra, node_to_bb = parse_dotfiles(
    dotfiles,
    source_dir,
    debug_info_dict,
    blocks_dict,
    non_overlap_lineidx=False,
)

cfg_inter = gen_cfg_inter(cfgs_intra)
graph = Graph(cfg_inter)

map_fuzz_to_obs_node, map_obs_to_fuzz_node = get_mapping(
    functions, bbs_fuzz, node_to_bb, cfg_inter, debug=True
)


[afl_init_argv]
bb_fuzz_idxs=      67 node_obs=Node0x12e4106b0      ('afl_init_argv', 0)                obs_node_lines={48, 40, 45, 46} vs fuzz_node_lines=[40, 45, 46, 48]
bb_fuzz_idxs=      68 node_obs=Node0x12e411b60      ('afl_init_argv', 1)                obs_node_lines={48} vs fuzz_node_lines=[48]
bb_fuzz_idxs=      69 node_obs=Node0x12e411be0      ('afl_init_argv', 2)                obs_node_lines={50} vs fuzz_node_lines=[50]
bb_fuzz_idxs=      70 node_obs=Node0x12e4106f0      ('afl_init_argv', 3)                obs_node_lines={50} vs fuzz_node_lines=[50]
bb_fuzz_idxs=      71 node_obs=Node0x12e4123b0      ('afl_init_argv', 4)                obs_node_lines={52, 53} vs fuzz_node_lines=[52, 53]
bb_fuzz_idxs=      72 node_obs=Node0x12e413390      ('afl_init_argv', 5)                obs_node_lines={53} vs fuzz_node_lines=[53]
bb_fuzz_idxs=      73 node_obs=Node0x12e412a10      ('afl_init_argv', 6)                obs_node_lines={53} vs fuzz_node_lines=[53]
bb_fuzz_idxs=      74 node_o

In [7]:
projname = "schedule2"
runs_path = "fuzz-data/schedule2_exp_1"
run_paths = get_run_paths(runs_path, projname)
covs_trials_f = []
for run_path in run_paths:
    for trial_path in get_trial_paths(run_path):
        covs_trial = get_covs_trial(trial_path)
        covs = [cov for cov in covs_trial if max(cov) > 0]
        covs_trials_f.append(covs)
print(f"number of trials: {len(covs_trials_f)}")

number of trials: 990


In [8]:
covs_f = []
for covs in covs_trials_f:
    covs_f.extend(covs)
covs_f = np.array(covs_f)
print(f"{covs_f.shape=}")
cov_maxs = np.max(covs_f, axis=1)
print(f"{cov_maxs.shape=}")
get_statistics(cov_maxs)


covs_f.shape=(322981, 138)
cov_maxs.shape=(322981,)
mean: 209.6297119644809, std: 163.22375647498407, min: 1, max: 1405
quantiles:
          0
0.01    2.0
0.05   89.0
0.10   91.0
0.15   99.0
0.20  101.0
0.25  111.0
0.30  112.0
0.35  125.0
0.40  127.0
0.45  143.0
0.50  146.0
0.55  169.0
0.60  171.0
0.65  203.0
0.70  207.0
0.75  258.0
0.80  264.0
0.85  351.0
0.90  468.0
0.95  595.0
0.99  769.0


In [17]:
in_between_range = [89 <= cov_max_f <= 595 for cov_max_f in cov_maxs]
cov_data_f = covs_f[in_between_range]
print(f"{cov_data_f.shape=}")
sum_data_f = np.sum(cov_data_f, axis=0)
print(f"{sum_data_f.shape=}")
min_obs = np.min(sum_data_f[sum_data_f > 0])
print(f"{min_obs=}")
num_obss = cov_maxs[in_between_range]
total_obs = np.sum(num_obss)
print(f"{total_obs=}")

cov_data_f.shape=(291012, 138)
sum_data_f.shape=(138,)
min_obs=675309
total_obs=55585523
min prob: 675309/55585523 = 0.012149008654645563
* Note. np.max(sum_data_f)=55585487


In [18]:
nodenames = list(cfg_inter["nodes"].keys())
cov_data_o = fuzzdat_to_obs(
    cov_data_f, cfg_inter, bbs_fuzz, map_fuzz_to_obs_node, nodenames
)
print(f"{cov_data_o.shape=}")
sum_data_o = np.sum(cov_data_o, axis=0)
print(f"{sum_data_o.shape=}")

cov_data_o.shape=(291012, 302)
sum_data_o.shape=(302,)


In [19]:
def order_found(
    covs: np.ndarray, observable_ids_f, consider_remaining: bool = True
):
    founded = set()
    ret = {idx: np.nan for idx in range(len(covs[0]))}
    last_found = {}
    for cov in covs:
        found = {idx for idx in range(len(cov)) if cov[idx] > 0}
        new_found = found - founded
        founded = founded | found
        for idx in new_found:
            ret[idx] = len(founded)
        if len(new_found):
            last_found = new_found
    if consider_remaining:
        remaining = observable_ids_f - founded
        if len(remaining):
            founded = founded | remaining
            for idx in remaining:
                ret[idx] = len(founded)
            last_found = remaining
    return ret, last_found

In [20]:
observable_ids_f = {i for i in range(len(sum_data_f)) if sum_data_f[i] > 0}
order_data = {idx: [] for idx in range(len(covs_trials_f[0][0]))}
last_found_cnt = {}
for covs_trial in covs_trials_f:
    order_trial, last_found = order_found(covs_trial, observable_ids_f, consider_remaining=False)
    for idx in order_trial:
        if np.isnan(order_trial[idx]):
            continue
        order_data[idx].append(order_trial[idx])
    for idx in last_found:
        if idx not in last_found_cnt:
            last_found_cnt[idx] = 0
        last_found_cnt[idx] += 1
for idx, orders in order_data.items():
    if idx <= 10:
        print(f"{idx}: {np.nanmean(orders):.2f} ({orders[:5]}...)")
    order_data[idx] = np.nanmean(order_data[idx])
    if np.isnan(order_data[idx]):
        order_data[idx] = 0

order_df = pd.DataFrame(order_data, index=[0]).T
display(order_df.sort_values(by=0, ascending=False).head(10))

last_found_df = pd.DataFrame(last_found_cnt, index=[0])
display(last_found_df.T.sort_values(by=0, ascending=False).head(10))

0: 72.65 ([70, 75, 70, 70, 75]...)
1: nan ([]...)
2: 72.65 ([70, 75, 70, 70, 75]...)
3: 72.65 ([70, 75, 70, 70, 75]...)
4: 72.65 ([70, 75, 70, 70, 75]...)
5: 72.65 ([70, 75, 70, 70, 75]...)
6: nan ([]...)
7: 72.65 ([70, 75, 70, 70, 75]...)
8: 72.65 ([70, 75, 70, 70, 75]...)
9: 73.60 ([70, 75, 70, 70, 75]...)
10: 73.60 ([70, 75, 70, 70, 75]...)


/var/folders/v9/wc5dh1fd691947y71hyt_2280000gn/T/ipykernel_89450/3607526601.py:16: RuntimeWarning: Mean of empty slice
  print(f"{idx}: {np.nanmean(orders):.2f} ({orders[:5]}...)")
/var/folders/v9/wc5dh1fd691947y71hyt_2280000gn/T/ipykernel_89450/3607526601.py:17: RuntimeWarning: Mean of empty slice
  order_data[idx] = np.nanmean(order_data[idx])


,0
52,75.455257
23,74.586869
22,74.586869
43,74.586869
33,74.586869
37,74.586869
9,73.595960
10,73.595960
14,73.289899
0,72.645455


,0
52,868
37,102
43,102
22,102
23,102
33,102
9,79
10,79
14,68
41,49


In [21]:
ids_of_interest_f = [52]
for idx_f in ids_of_interest_f:
    print(
        f"map_fuzz_to_obs_node[{idx_f}]={map_fuzz_to_obs_node[idx_f]}"
    )


map_fuzz_to_obs_node[52]={'Node0x12e40c1d0-0', 'Node0x12e40c1d0-1'}


# Totinfo

In [ ]:
source_dir = "fuzz-data/ft_data/source_code/totinfo"
source_name = "totinfo_fuzz"

dotfiles = [
    fname
    for fname in os.listdir(source_dir)
    if fname.startswith(".") and fname.endswith(".dot")
]
functions = sorted([fname.split(".")[1] for fname in dotfiles])

bbs_fuzz_path = "fuzz-data/totinfo_exp_1/totinfo_01/totinfo_aflpp_run_01/ft_totinfo.json"
bbs_fuzz = get_bbs_fuzz(bbs_fuzz_path)

debug_info_dict, blocks_dict = parse_ll(
    "fuzz-data/ft_data/source_code/totinfo/totinfo_fuzz.ll"
)
cfgs_intra, node_to_bb = parse_dotfiles(
    dotfiles,
    source_dir,
    debug_info_dict,
    blocks_dict,
    non_overlap_lineidx=False,
)

cfg_inter = gen_cfg_inter(cfgs_intra)
graph = Graph(cfg_inter)

map_fuzz_to_obs_node, map_obs_to_fuzz_node = get_mapping(
    functions, bbs_fuzz, node_to_bb, cfg_inter, debug=True
)


In [ ]:
projname = "totinfo"
runs_path = "fuzz-data/totinfo_exp_1"
run_paths = get_run_paths(runs_path, projname)
covs_trials_f = []
for run_path in run_paths:
    for trial_path in get_trial_paths(run_path):
        covs_trial = get_covs_trial(trial_path)
        covs = [cov for cov in covs_trial if max(cov) > 0]
        covs_trials_f.append(covs)
print(f"number of trials: {len(covs_trials_f)}")

In [ ]:
covs_f = []
for covs in covs_trials_f:
    covs_f.extend(covs)
covs_f = np.array(covs_f)
print(f"{covs_f.shape=}")
cov_maxs = np.max(covs_f, axis=1)
print(f"{cov_maxs.shape=}")
get_statistics(cov_maxs)


In [ ]:
in_between_range = [32 <= cov_max_f <= 524 for cov_max_f in cov_maxs]
cov_data_f = covs_f[in_between_range]
print(f"{cov_data_f.shape=}")
sum_data_f = np.sum(cov_data_f, axis=0)
print(f"{sum_data_f.shape=}")
min_obs = np.min(sum_data_f[sum_data_f > 0])
print(f"{min_obs=}")
num_obss = cov_maxs[in_between_range]
total_obs = np.sum(num_obss)
print(f"{total_obs=}")


In [ ]:
nodenames = list(cfg_inter["nodes"].keys())
cov_data_o = fuzzdat_to_obs(
    cov_data_f, cfg_inter, bbs_fuzz, map_fuzz_to_obs_node, nodenames
)
print(f"{cov_data_o.shape=}")
sum_data_o = np.sum(cov_data_o, axis=0)
print(f"{sum_data_o.shape=}")


In [ ]:
observable_ids_f = {i for i in range(len(sum_data_f)) if sum_data_f[i] > 0}

order_data = {idx: [] for idx in range(len(covs_trials_f[0][0]))}
last_found_cnt = {}
for covs_trial in covs_trials_f:
    order_trial, last_found = order_found(covs_trial, observable_ids_f, consider_remaining=False)
    for idx in order_trial:
        if np.isnan(order_trial[idx]):
            continue
        order_data[idx].append(order_trial[idx])
    for idx in last_found:
        if idx not in last_found_cnt:
            last_found_cnt[idx] = 0
        last_found_cnt[idx] += 1
for idx, orders in order_data.items():
    if idx <= 10:
        print(f"{idx}: {np.nanmean(orders):.2f} ({orders[:5]}...)")
    order_data[idx] = np.nanmean(order_data[idx])
    if np.isnan(order_data[idx]):
        order_data[idx] = 0

order_df = pd.DataFrame(order_data, index=[0]).T
display(order_df.sort_values(by=0, ascending=False).head(10))

last_found_df = pd.DataFrame(last_found_cnt, index=[0])
display(last_found_df.T.sort_values(by=0, ascending=False).head(10))

In [ ]:
ids_of_interest_f = [29, 125, 52, 15, 27, 81]
for idx_f in ids_of_interest_f:
    print(
        f"map_fuzz_to_obs_node[{idx_f}]={map_fuzz_to_obs_node[idx_f]}"
    )


# Jsoncpp

In [ ]:
source_dir = "fuzz-data/ft_data/llvm/jsoncpp"
source_name = "jsoncpp"

dotfiles = [
    fname
    for fname in os.listdir(source_dir)
    if fname.startswith(".") and fname.endswith(".dot")
]
functions = sorted([fname.split(".")[1] for fname in dotfiles])

bbs_fuzz_path = "fuzz-data/jsoncpp-fuzzruns/jsoncpp_01/jsoncpp_aflpp_seed_01_run_01/ft_jsoncpp.json"
bbs_fuzz = get_bbs_fuzz(bbs_fuzz_path)

debug_info_dict, blocks_dict = parse_ll(
    os.path.join(source_dir, f"{source_name}.ll")
)
cfgs_intra, node_to_bb = parse_dotfiles(
    dotfiles,
    source_dir,
    debug_info_dict,
    blocks_dict,
    non_overlap_lineidx=False,
)

cfg_inter = gen_cfg_inter(cfgs_intra)
graph = Graph(cfg_inter, start_func="_ZN4Json13OurCharReader5parseEPKcS2_PNS_5ValueEPNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE")

map_fuzz_to_obs_node, map_obs_to_fuzz_node = get_mapping(
    functions, bbs_fuzz, node_to_bb, cfg_inter, debug=True
)


In [ ]:
projname = "jsoncpp"
runs_path = "fuzz-data/jsoncpp-fuzzruns"
run_paths = get_run_paths(runs_path, projname)
covs_trials_f = []
for run_path in run_paths:
    for trial_path in get_trial_paths(run_path):
        covs_trial = get_covs_trial(
            trial_path, expected_time_interval=(899, 902)
        )
        covs = [cov for cov in covs_trial if max(cov) > 0]
        covs_trials_f.append(covs)
print(f"number of trials: {len(covs_trials_f)}")


In [ ]:
covs_f = []
for covs in covs_trials_f:
    covs_f.extend(covs)
covs_f = np.array(covs_f)
print(f"{covs_f.shape=}")
cov_maxs = np.max(covs_f, axis=1)
print(f"{cov_maxs.shape=}")
get_statistics(cov_maxs)


In [ ]:
in_between_range = [
    115058.85 <= cov_max_f <= 463113.65 for cov_max_f in cov_maxs
]
cov_data_f = covs_f[in_between_range]
print(f"{cov_data_f.shape=}")
sum_data_f = np.sum(cov_data_f, axis=0)
print(f"{sum_data_f.shape=}")
min_obs = np.min(sum_data_f[sum_data_f > 0])
print(f"{min_obs=}")
num_obss = cov_maxs[in_between_range]
total_obs = np.sum(num_obss)
print(f"{total_obs=}")


In [ ]:
nodenames = list(cfg_inter["nodes"].keys())
cov_data_o = fuzzdat_to_obs(
    cov_data_f, cfg_inter, bbs_fuzz, map_fuzz_to_obs_node, nodenames
)
np.save("notepad/hard-to-reach-node/jsoncpp-cov-data-o.npy", cov_data_o)
# cov_data_o = np.load("notepad/hard-to-reach-node/jsoncpp-cov-data-o.npy")
print(f"{cov_data_o.shape=}")
sum_data_o = np.sum(cov_data_o, axis=0)
print(f"{sum_data_o.shape=}")


In [ ]:
observable_ids_f = {i for i in range(len(sum_data_f)) if sum_data_f[i] > 0}

order_data = {idx: [] for idx in range(len(covs_trials_f[0][0]))}
last_found_cnt = {}
for covs_trial in covs_trials_f:
    order_trial, last_found = order_found(covs_trial, observable_ids_f, consider_remaining=False)
    for idx in order_trial:
        if np.isnan(order_trial[idx]):
            continue
        order_data[idx].append(order_trial[idx])
    for idx in last_found:
        if idx not in last_found_cnt:
            last_found_cnt[idx] = 0
        last_found_cnt[idx] += 1
for idx, orders in order_data.items():
    if idx <= 10:
        print(f"{idx}: {np.nanmean(orders):.2f} ({orders[:5]}...)")
    order_data[idx] = np.nanmean(order_data[idx])
    if np.isnan(order_data[idx]):
        order_data[idx] = 0

order_df = pd.DataFrame(order_data, index=[0]).T
display(order_df.sort_values(by=0, ascending=False).head(10))

last_found_df = pd.DataFrame(last_found_cnt, index=[0])
display(last_found_df.T.sort_values(by=0, ascending=False).head(10))

In [ ]:
ids_of_interest_f = [
    360,
    361,
    362,
    363,
    364,
    365,
    359,
    729,
    730,
    765,
    766,
    928,
    926,
    931,
    729,
    927,
    730,
    934,
    933,
    932
]
for idx_f in ids_of_interest_f:
    if idx_f not in map_fuzz_to_obs_node:
        print(f"idx_f={idx_f} not in map_fuzz_to_obs_node")
        continue
    print(
        f"map_fuzz_to_obs_node[{idx_f}]={map_fuzz_to_obs_node[idx_f]}"
    )


# Readelf

In [ ]:
source_dir = "fuzz-data/ft_data/llvm/readelf"
source_name = "readelf"

dotfiles = [
    fname
    for fname in os.listdir(source_dir)
    if fname.startswith(".") and fname.endswith(".dot")
]
functions = sorted([fname.split(".")[1] for fname in dotfiles])

bbs_fuzz_path = "fuzz-data/readelf-fuzzruns/readelf_01/readelf_aflpp_seed_01_run_01/ft_readelf.json"
bbs_fuzz = get_bbs_fuzz(bbs_fuzz_path)

debug_info_dict, blocks_dict = parse_ll(
    os.path.join(source_dir, f"{source_name}.ll")
)
cfgs_intra, node_to_bb = parse_dotfiles(
    dotfiles,
    source_dir,
    debug_info_dict,
    blocks_dict,
    non_overlap_lineidx=False,
)

cfg_inter = gen_cfg_inter(cfgs_intra)
graph = Graph(cfg_inter)

map_fuzz_to_obs_node, map_obs_to_fuzz_node = get_mapping(
    functions, bbs_fuzz, node_to_bb, cfg_inter, debug=True
)


In [ ]:
projname = "readelf"
runs_path = "fuzz-data/readelf-fuzzruns"
run_paths = get_run_paths(runs_path, projname)
covs_trials_f = []
for run_path in run_paths:
    for trial_path in get_trial_paths(run_path):
        covs_trial = get_covs_trial(
            trial_path, expected_time_interval=(899, 902)
        )
        covs = [cov for cov in covs_trial if max(cov) > 0]
        covs_trials_f.append(covs)
print(f"number of trials: {len(covs_trials_f)}")


In [ ]:
covs_f = []
for covs in covs_trials_f:
    covs_f.extend(covs)
covs_f = np.array(covs_f)
print(f"{covs_f.shape=}")
cov_maxs = np.max(covs_f, axis=1)
print(f"{cov_maxs.shape=}")
get_statistics(cov_maxs)


In [ ]:
in_between_range = [
    159484.05 <= cov_max_f <= 218819.20 for cov_max_f in cov_maxs
]
cov_data_f = covs_f[in_between_range]
print(f"{cov_data_f.shape=}")
sum_data_f = np.sum(cov_data_f, axis=0)
print(f"{sum_data_f.shape=}")
min_obs = np.min(sum_data_f[sum_data_f > 0])
print(f"{min_obs=}")
num_obss = cov_maxs[in_between_range]
total_obs = np.sum(num_obss)
print(f"{total_obs=}")


In [ ]:
nodenames = list(cfg_inter["nodes"].keys())
cov_data_o = fuzzdat_to_obs(
    cov_data_f, cfg_inter, bbs_fuzz, map_fuzz_to_obs_node, nodenames
)
np.save("notepad/hard-to-reach-node/readelf-cov-data-o.npy", cov_data_o)
# cov_data_o = np.load("notepad/hard-to-reach-node/readelf-cov-data-o.npy")
print(f"{cov_data_o.shape=}")
sum_data_o = np.sum(cov_data_o, axis=0)
print(f"{sum_data_o.shape=}")


In [ ]:
observable_ids_f = {i for i in range(len(sum_data_f)) if sum_data_f[i] > 0}

order_data = {idx: [] for idx in range(len(covs_trials_f[0][0]))}
last_found_cnt = {}
for covs_trial in covs_trials_f:
    order_trial, last_found = order_found(covs_trial, observable_ids_f, consider_remaining=False)
    for idx in order_trial:
        if np.isnan(order_trial[idx]):
            continue
        order_data[idx].append(order_trial[idx])
    for idx in last_found:
        if idx not in last_found_cnt:
            last_found_cnt[idx] = 0
        last_found_cnt[idx] += 1
for idx, orders in order_data.items():
    if idx <= 10:
        print(f"{idx}: {np.nanmean(orders):.2f} ({orders[:5]}...)")
    order_data[idx] = np.nanmean(order_data[idx])
    if np.isnan(order_data[idx]):
        order_data[idx] = 0

order_df = pd.DataFrame(order_data, index=[0]).T
display(order_df.sort_values(by=0, ascending=False).head(10))

last_found_df = pd.DataFrame(last_found_cnt, index=[0])
display(last_found_df.T.sort_values(by=0, ascending=False).head(10))

# Freetype2

In [ ]:
source_dir = "fuzz-data/ft_data/llvm/freetype2"
source_name = "freetype2"

dotfiles = [
    fname
    for fname in os.listdir(source_dir)
    if fname.startswith(".") and fname.endswith(".dot")
]
functions = sorted([fname.split(".")[1] for fname in dotfiles])

bbs_fuzz_path = "fuzz-data/freetype2-fuzzruns/freetype2_01/freetype2_aflpp_seed_01_run_01/ft_freetype2.json"
bbs_fuzz = get_bbs_fuzz(bbs_fuzz_path)

debug_info_dict, blocks_dict = parse_ll(
    os.path.join(source_dir, f"{source_name}.ll")
)
cfgs_intra, node_to_bb = parse_dotfiles(
    dotfiles,
    source_dir,
    debug_info_dict,
    blocks_dict,
    non_overlap_lineidx=False,
)

cfg_inter = gen_cfg_inter(cfgs_intra)
graph = Graph(cfg_inter, start_func="FT_New_Memory_Face")

map_fuzz_to_obs_node, map_obs_to_fuzz_node = get_mapping(
    functions, bbs_fuzz, node_to_bb, cfg_inter, debug=True
)


In [ ]:
projname = "freetype2"
runs_path = "fuzz-data/freetype2-fuzzruns"
run_paths = get_run_paths(runs_path, projname)
covs_trials_f = []
for run_path in run_paths:
    for trial_path in get_trial_paths(run_path):
        covs_trial = get_covs_trial(
            trial_path, expected_time_interval=(899, 902)
        )
        covs = [cov for cov in covs_trial if max(cov) > 0]
        covs_trials_f.append(covs)
print(f"number of trials: {len(covs_trials_f)}")


In [ ]:
covs_f = []
for covs in covs_trials_f:
    covs_f.extend(covs)
covs_f = np.array(covs_f)
print(f"{covs_f.shape=}")
cov_maxs = np.max(covs_f, axis=1)
print(f"{cov_maxs.shape=}")
get_statistics(cov_maxs)


In [ ]:
in_between_range = [
    267558.45 <= cov_max_f <= 327647.65 for cov_max_f in cov_maxs
]
cov_data_f = covs_f[in_between_range]
print(f"{cov_data_f.shape=}")
sum_data_f = np.sum(cov_data_f, axis=0)
print(f"{sum_data_f.shape=}")
min_obs = np.min(sum_data_f[sum_data_f > 0])
print(f"{min_obs=}")
num_obss = cov_maxs[in_between_range]
total_obs = np.sum(num_obss)
print(f"{total_obs=}")


In [ ]:
nodenames = list(cfg_inter["nodes"].keys())
cov_data_o = fuzzdat_to_obs(
    cov_data_f, cfg_inter, bbs_fuzz, map_fuzz_to_obs_node, nodenames
)
np.save("notepad/hard-to-reach-node/freetype2-cov-data-o.npy", cov_data_o)
# cov_data_o = np.load("notepad/hard-to-reach-node/freetype2-cov-data-o.npy")
print(f"{cov_data_o.shape=}")
sum_data_o = np.sum(cov_data_o, axis=0)
print(f"{sum_data_o.shape=}")


In [ ]:
observable_ids_f = {i for i in range(len(sum_data_f)) if sum_data_f[i] > 0}

order_data = {idx: [] for idx in range(len(covs_trials_f[0][0]))}
last_found_cnt = {}
for covs_trial in covs_trials_f:
    order_trial, last_found = order_found(covs_trial, observable_ids_f, consider_remaining=False)
    for idx in order_trial:
        if np.isnan(order_trial[idx]):
            continue
        order_data[idx].append(order_trial[idx])
    for idx in last_found:
        if idx not in last_found_cnt:
            last_found_cnt[idx] = 0
        last_found_cnt[idx] += 1
for idx, orders in order_data.items():
    if idx <= 10:
        print(f"{idx}: {np.nanmean(orders):.2f} ({orders[:5]}...)")
    order_data[idx] = np.nanmean(order_data[idx])
    if np.isnan(order_data[idx]):
        order_data[idx] = 0

order_df = pd.DataFrame(order_data, index=[0]).T
display(order_df.sort_values(by=0, ascending=False).head(10))

last_found_df = pd.DataFrame(last_found_cnt, index=[0])
display(last_found_df.T.sort_values(by=0, ascending=False).head(10))

In [ ]:
ids_of_interest_f = [670, 672, 18736, 2186, 18667, 2188, 2189, 2190, 2191, 2192]
for idx_f in ids_of_interest_f:
    if idx_f not in map_fuzz_to_obs_node:
        print(f"idx_f={idx_f} not in map_fuzz_to_obs_node")
        continue
    print(
        f"map_fuzz_to_obs_node[{idx_f}]={map_fuzz_to_obs_node[idx_f]}"
    )
